In [23]:
from tqdm import tqdm
import torch
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader
from DiacriticDataset import DiacriticDatasetShaddah
import pickle
from collections import Counter
from itertools import product
from collections import defaultdict
import numpy as np

In [6]:
test_dataset = DiacriticDatasetShaddah('Dataset_with_shaddah/test_cleaned_withshadda_215.txt','Dataset_with_shaddah/letter_to_id.pickle','Dataset_with_shaddah/id_to_letter.pickle','Dataset_with_shaddah/diacritic_to_id.pickle','Dataset_with_shaddah/id_to_diacritic.pickle','Dataset_with_shaddah/word_to_id.pickle','Dataset_with_shaddah/id_to_word.pickle','Dataset/diacritic_to_id.pickle','Dataset/id_to_diacritic.pickle','Dataset_with_shaddah/word_to_id_withdiacritics.pickle','Dataset_with_shaddah/id_to_word_withdiacritics.pickle')

In [7]:
test_dataloader = DataLoader(test_dataset)

In [16]:
def viterbi(letter_sequence,q,e):
    """
    :param letter_sequence:List  A sequence of letter ids
    :param q: transition probabilities
    :param e: emission probabilities
    :return:List a sequence of tag ids
    """
    pi = {}
    bp = {}
    pi[(-1,'*','*')] = 1
    num_of_letters = len(letter_sequence)
    tag_sequence = [0]*num_of_letters
    for k in range(num_of_letters):
        if k==0:
            s_u = ['*']
            s_v = list(range(16))
            s_w =['*']
        elif k==1:
            s_u = list(range(16))
            s_v = list(range(16))
            s_w = ['*']
        else:
            s_u = list(range(16))
            s_v = list(range(16))
            s_w = list(range(16))
            
        for u in s_u:
            for v in s_v:
                probs = []
                for w in s_w:
                    probs.append(pi[(k-1,w,u)]*q[v,w,u]*e[letter_sequence[k],v])
                    
                max_prob = max(probs)
                pi[(k,u,v)] = max_prob
                bp[(k,u,v)] = s_w[probs.index(max_prob)]
                
    max_u = 0
    max_v = 0
    max_value = 0
    s = list(range(16))
    for u in s:
        for v in s:
            if (pi[(num_of_letters-1,u,v)]*q['STOP',u,v])>max_value:
                max_value = (pi[(num_of_letters-1,u,v)]*q['STOP',u,v])
                max_u = u
                max_v = v
    tag_sequence[num_of_letters-1] = max_v
    tag_sequence[num_of_letters-2] = max_u

    for k in reversed(range(num_of_letters-2)):
        tag_sequence[k] = bp[(k+2,tag_sequence[k+1],tag_sequence[k+2])]


    return tag_sequence

In [35]:
with open('HMM_Model/transition_probabilities','rb') as file:
    transitions = pickle.load(file)
    
with open('HMM_Model/emission_probabilities','rb') as file:
    emissions = pickle.load(file)

tp = 0
total = 0
    
tp_word = 0
total_word = 0

wrong_words = []

for i,test_example in enumerate(tqdm(test_dataloader)):
    letters = test_example[0][0].tolist()
    true_diacritics = test_example[1][0].tolist()
    true_words = test_example[4][0].tolist()
    predicted_diacritics = viterbi(letters,transitions,emissions)
    
    tp += (np.array(true_diacritics)==np.array(predicted_diacritics)).sum()
    total += len(true_diacritics)
    
    sentence = ''
    for index,letter_id in enumerate(letters):
        letter = test_dataset.id_to_letter[letter_id]
        diacritic = test_dataset.id_to_diacritic[predicted_diacritics[index]]
        if letter_id==2:
            sentence += " "
            diacritic = ""
        elif predicted_diacritics[index] == 3: 
            diacritic = ""

        sentence += (letter+diacritic)


    predicted_words = sentence.split()

    for index,predicted_word in enumerate(predicted_words):
        predicted_word_id = test_dataset.word_to_id_diacs.get(predicted_word,'NOT_FOUND')
        if predicted_word_id != 'NOT_FOUND':
            if predicted_word_id == true_words[index]:
                tp_word += 1
            else:
                wrong_words.append(test_dataset.id_to_word_diacs[true_words[index]])
        else:
            wrong_words.append(test_dataset.id_to_word_diacs[true_words[index]])
            continue


    total_word += len(predicted_words)


diacritic_accuracy = tp/total
word_accuracy = tp_word/total_word
print("Test Diacritic Accuracy: {:.3f}\tTest Word Accuracy: {:3f}".format(diacritic_accuracy,word_accuracy))

100%|██████████| 1650/1650 [05:23<00:00,  5.09it/s]

Test Diacritic Accuracy: 0.629	Test Word Accuracy: 0.174783


In [39]:
total

142905